In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println("Error in $file")
        end
    end
end

Error in 45_rwm_wip_2_6r7m.jld2


In [4]:
argmin(min_Es)

37

In [5]:
for elem in evaluation_strings
    println(elem)
end


2: 2.5 | 0.0615239 | 8167.740043099553, 8033.951283430558, 8033.951283430558 | 30.440251705183528
3: 2.5 | 0.034963768 | 8149.489619615257, 8031.353001035705, 8031.353001035705 | 28.53389852849882
4: 2.5 | 0.024230108 | 8159.969944462297, 8055.90730642551, 8069.134295755852 | 37.09565786678185
5: 2.5 | 0.10775669 | 8129.668027120584, 8083.068484558059, 8094.249539752909 | 104.6314116220699
6: 2.5 | 0.011034184 | 8105.645860366124, 8027.557741383568, 8031.718704402193 | 16.85968036630731
7: 2.5 | 0.06133916 | 8164.211600952591, 8025.144351990336, 8028.896940737338 | 43.24335143215215
8: 2.5 | 0.13020678 | 8145.622537955674, 8081.923615353447, 8094.521791219532 | 71.730130070915
9: 2.5 | 0.13046098 | 8153.222028554101, 8051.207534799093, 8056.381825394533 | 28.948380733066642
10: 2.5 | 0.05657345 | 8133.964137121024, 8045.5321042748665, 8052.607445699947 | 22.85539980667812
11: 2.5 | 0.10377496 | 8167.832205824971, 8079.311346012141, 8099.8814393284265 | 104.75657306630225
12: 2.5 | 0.0

In [6]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [34]:
@load "../../Data/hpc_out/rwm_op_3_6r7m/5_rwm_op_3_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [35]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [36]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
